In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile6 = pd.read_pickle("DDD_Train_Decile6.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile6.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl")
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile6, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones',
                moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None,
                beta_constraint=None, gamma_constraint=None))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,75,100,125,150])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 6')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

2264/2264 [==============================] - 9s 4ms/step - loss: 0.4190 - acc: 0.8127 - val_loss: 0.3812 - val_acc: 0.8413
Epoch 36/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4259 - acc: 0.8145 - val_loss: 0.3814 - val_acc: 0.8413
Epoch 37/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4210 - acc: 0.8176 - val_loss: 0.3800 - val_acc: 0.8430
Epoch 38/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4199 - acc: 0.8158 - val_loss: 0.3773 - val_acc: 0.8430
Epoch 39/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4109 - acc: 0.8207 - val_loss: 0.3755 - val_acc: 0.8466
Epoch 40/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4218 - acc: 0.8167 - val_loss: 0.3768 - val_acc: 0.8448
Epoch 41/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4151 - acc: 0.8233 - val_loss: 0.3766 - val_acc: 0.8448
Epoch 42/75
2264/2264 [==============================] - 8s 4ms/ste

2264/2264 [==============================] - 8s 4ms/step - loss: 0.3666 - acc: 0.8414 - val_loss: 0.3258 - val_acc: 0.8483
Epoch 21/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3636 - acc: 0.8379 - val_loss: 0.3302 - val_acc: 0.8448
Epoch 22/75
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3696 - acc: 0.8366 - val_loss: 0.3297 - val_acc: 0.8413
Epoch 23/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3682 - acc: 0.8383 - val_loss: 0.3279 - val_acc: 0.8483

Epoch 00023: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 24/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3627 - acc: 0.8428 - val_loss: 0.3266 - val_acc: 0.8466
Epoch 25/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3614 - acc: 0.8436 - val_loss: 0.3278 - val_acc: 0.8448
Epoch 26/75
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3642 - acc: 0.8441 - val_loss: 0.3274 - v

2264/2264 [==============================] - 9s 4ms/step - loss: 0.4717 - acc: 0.7973 - val_loss: 0.4106 - val_acc: 0.8395
Epoch 4/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4435 - acc: 0.8145 - val_loss: 0.3932 - val_acc: 0.8483
Epoch 5/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4291 - acc: 0.8070 - val_loss: 0.3883 - val_acc: 0.8448
Epoch 6/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.4239 - acc: 0.8171 - val_loss: 0.3827 - val_acc: 0.8448
Epoch 7/100
2264/2264 [==============================] - 6s 3ms/step - loss: 0.4207 - acc: 0.8189 - val_loss: 0.3763 - val_acc: 0.8395
Epoch 8/100
2264/2264 [==============================] - 7s 3ms/step - loss: 0.4106 - acc: 0.8145 - val_loss: 0.3659 - val_acc: 0.8413
Epoch 9/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4043 - acc: 0.8238 - val_loss: 0.3665 - val_acc: 0.8395
Epoch 10/100
2264/2264 [==============================] - 9s 4ms/st

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3715 - acc: 0.8397 - val_loss: 0.3248 - val_acc: 0.8430
Epoch 63/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3594 - acc: 0.8423 - val_loss: 0.3247 - val_acc: 0.8448
Epoch 64/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3681 - acc: 0.8335 - val_loss: 0.3246 - val_acc: 0.8448
Epoch 65/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3623 - acc: 0.8383 - val_loss: 0.3246 - val_acc: 0.8448
Epoch 66/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3731 - acc: 0.8330 - val_loss: 0.3246 - val_acc: 0.8430
Epoch 67/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3639 - acc: 0.8299 - val_loss: 0.3247 - val_acc: 0.8430
Epoch 68/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3728 - acc: 0.8379 - val_loss: 0.3247 - val_acc: 0.8430
Epoch 69/100
2264/2264 [==============================] - 9s 

...

2264/2264 [==============================] - 9s 4ms/step - loss: 0.4331 - acc: 0.8163 - val_loss: 0.4019 - val_acc: 0.8342
Epoch 27/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4348 - acc: 0.8087 - val_loss: 0.4000 - val_acc: 0.8360
Epoch 28/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4375 - acc: 0.8149 - val_loss: 0.3988 - val_acc: 0.8342
Epoch 29/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4267 - acc: 0.8149 - val_loss: 0.3974 - val_acc: 0.8342
Epoch 30/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4326 - acc: 0.8017 - val_loss: 0.3964 - val_acc: 0.8342
Epoch 31/125
2264/2264 [==============================] - 10s 4ms/step - loss: 0.4308 - acc: 0.8158 - val_loss: 0.3952 - val_acc: 0.8342
Epoch 32/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4298 - acc: 0.8052 - val_loss: 0.3945 - val_acc: 0.8325
Epoch 33/125
2264/2264 [==============================] - 9s

Epoch 86/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4071 - acc: 0.8224 - val_loss: 0.3706 - val_acc: 0.8483
Epoch 87/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4029 - acc: 0.8242 - val_loss: 0.3706 - val_acc: 0.8466
Epoch 88/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4020 - acc: 0.8242 - val_loss: 0.3705 - val_acc: 0.8466
Epoch 89/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4076 - acc: 0.8127 - val_loss: 0.3702 - val_acc: 0.8466
Epoch 90/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4020 - acc: 0.8118 - val_loss: 0.3701 - val_acc: 0.8448
Epoch 91/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4080 - acc: 0.8193 - val_loss: 0.3701 - val_acc: 0.8466
Epoch 92/125
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4055 - acc: 0.8171 - val_loss: 0.3701 - val_acc: 0.8466
Epoch 93/125
2264/2264 [========================

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3754 - acc: 0.8361 - val_loss: 0.3215 - val_acc: 0.8448
Epoch 21/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.4040 - acc: 0.8269 - val_loss: 0.3449 - val_acc: 0.8395
Epoch 22/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3892 - acc: 0.8299 - val_loss: 0.3286 - val_acc: 0.8430
Epoch 23/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3661 - acc: 0.8370 - val_loss: 0.3270 - val_acc: 0.8430
Epoch 24/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3677 - acc: 0.8344 - val_loss: 0.3281 - val_acc: 0.8413
Epoch 25/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3657 - acc: 0.8322 - val_loss: 0.3252 - val_acc: 0.8413

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 26/100
2264/2264 [==============================] - 8s 4ms/step - loss: 0.3650 - acc: 0.8330 - val_loss: 0.32

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3664 - acc: 0.8344 - val_loss: 0.3235 - val_acc: 0.8413
Epoch 79/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3589 - acc: 0.8361 - val_loss: 0.3233 - val_acc: 0.8413
Epoch 80/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3560 - acc: 0.8379 - val_loss: 0.3233 - val_acc: 0.8413
Epoch 81/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3660 - acc: 0.8401 - val_loss: 0.3234 - val_acc: 0.8413
Epoch 82/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3549 - acc: 0.8392 - val_loss: 0.3233 - val_acc: 0.8413
Epoch 83/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3567 - acc: 0.8388 - val_loss: 0.3232 - val_acc: 0.8413
Epoch 84/100
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3610 - acc: 0.8414 - val_loss: 0.3234 - val_acc: 0.8413
Epoch 85/100
2264/2264 [==============================] - 9s 

2264/2264 [==============================] - 9s 4ms/step - loss: 0.3615 - acc: 0.8379 - val_loss: 0.3293 - val_acc: 0.8377
Epoch 38/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3598 - acc: 0.8370 - val_loss: 0.3290 - val_acc: 0.8377
Epoch 39/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3675 - acc: 0.8383 - val_loss: 0.3285 - val_acc: 0.8377
Epoch 40/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3606 - acc: 0.8388 - val_loss: 0.3277 - val_acc: 0.8377

Epoch 00040: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 41/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3635 - acc: 0.8441 - val_loss: 0.3276 - val_acc: 0.8377
Epoch 42/50
2264/2264 [==============================] - 10s 4ms/step - loss: 0.3642 - acc: 0.8432 - val_loss: 0.3277 - val_acc: 0.8377
Epoch 43/50
2264/2264 [==============================] - 9s 4ms/step - loss: 0.3680 - acc: 0.8295 - val_loss: 0.3277 - val_acc: 0.8377
E

In [ ]:
0.86434